In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser

faq_texts = [
    "The smartwatch battery lasts up to 7 days on a single charge.",
    "You can track your heart rate, sleep, and steps with the watch.",
    "The smartwatch is water-resistant up to 50 meters."
]
embedding = OpenAIEmbeddings()
vectorstore = DocArrayInMemorySearch.from_texts(faq_texts, embedding=embedding)
retriever = vectorstore.as_retriever()
template = """You are a helpful assistant answering questions based only on the following product FAQs:
{context}
Customer Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
output_parser = StrOutputParser()
chain = (
    RunnableMap({
        "context": lambda x: retriever.get_relevant_documents(x["question"]),
        "question": lambda x: x["question"]
    })| prompt| model| output_parser)

response = chain.invoke({"question": "Explain the features of the watch?"})
print(response)


The watch allows you to track your heart rate, sleep, and steps. It is water-resistant up to 50 meters, making it suitable for swimming and other water activities. The battery lasts up to 7 days on a single charge, providing long-lasting use without frequent recharging.
